In [2]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics

In [ ]:
# Station information
stations_info = CSV.read("../../data/stations/station_information.csv", DataFrame);
# Capacities
C = stations_info[!, :capacity];
# Distances
D = load("../../data/parameters/distances.jld2", "D");
# Feasibility
X = load("../../data/parameters/feasibility.jld2", "X");
# demand
demand = load("../../data/parameters/202210-demand.jld2", "demand");

In [ ]:
# Initial state 
y0 = floor.(C/2);
# demand over one day
# Day 1
d = demand[:,:,1:24];
# Day 8
d_2 = demand[:,:,169:192];
# Day 15
d_3 = demand[:,:,337:360];

In [ ]:
function reduce_problem(d,C,y0,D,X,nb)
    n_stations = size(d,1)
    stations_score = zeros(n_stations)
    # for each station i, sum the number of bikes that gets in an out ot the station over the month
    for i in 1:n_stations
        stations_score[i] += sum(d[i,:,:]) + sum(d[:,i,:])
    end
    # sort stations by score
    stations_score_sorted = sortperm(stations_score, rev=true);
    # reorder index of columns and rows according to the sorted stations
    d_sorted = d[stations_score_sorted, stations_score_sorted, :];
    C_sorted = C[stations_score_sorted,:];
    y0_sorted = y0[stations_score_sorted,:];
    D_sorted = D[stations_score_sorted, stations_score_sorted];
    X_sorted = X[stations_score_sorted, stations_score_sorted];
    # keep only the nb first stations
    d_new = d_sorted[1:nb,1:nb,:];
    C_new = C_sorted[1:nb,:];
    y0_new = y0_sorted[1:nb,:];
    D_new = D_sorted[1:nb,1:nb];
    X_new = X_sorted[1:nb,1:nb];
    return d_new, C_new, y0_new, D_new, X_new
end

In [ ]:
d_new, C_new, y0_new, D_new, X_new = reduce_problem(d,C,y0,D,X,200);

In [72]:
function solve_sequential_model(K,S,C,D,X,y0,d, lambda, n_rebal)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 100)

    n_stations, _, n_hours = size(d)
    M=30
    
    # Decision variables
    @variable(model, x[1:n_rebal, 1:n_rebal, 1:K, 1:n_hours], Bin)
    @variable(model, v[1:n_rebal, 1:n_rebal, 1:K, 1:n_hours], Bin)
    @variable(model, 0 <= z[1:n_rebal, 1:n_rebal, 1:K, 1:n_hours], Int)
    @variable(model, 0 <= y[1:n_stations, 1:n_hours]) # Int by definition of the constraints
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:n_hours], Int)
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:n_hours])

    # Add constraints:
    # Stations capacity
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], y[i,t] <= C[i])
    # No rebalancing at the same station
    @constraint(model, [i in 1:n_rebal, k in 1:K, t in 1:n_hours], x[i,i,k,t] == 0)
    # Flow balance 
    @constraint(model, [i in 1:n_rebal, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations)-sum(z[i,j,k,t] for j in 1:n_rebal, k in 1:K)+sum(z[j,i,k,t] for j in 1:n_rebal, k in 1:K))
    @constraint(model, [i in n_rebal+1:n_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations))
    # Flow balance for the first hour 
    @constraint(model, [i in 1:n_rebal], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations)-sum(z[i,j,k,1] for j in 1:n_rebal, k in 1:K)+sum(z[j,i,k,1] for j in 1:n_rebal, k in 1:K))
    @constraint(model, [i in n_rebal+1:n_stations], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations))
    # Vans capacity
    @constraint(model, [i in 1:n_rebal, j in 1:n_rebal, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= S)
    # Users travel availability
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations)<= C[i] - y[i,t])
    # Max 1 rebalancing per hour per van
    @constraint(model, [k in 1:K, t in 1:n_hours], sum(x[i,j,k,t] for i in 1:n_rebal, j in 1:n_rebal) <= 1)
    # Rebalancing feasibility
    @constraint(model, [i in 1:n_rebal, j in 1:n_rebal, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= M*x[i,j,k,t])
    @constraint(model, [i in 1:n_rebal, j in 1:n_rebal, k in 1:K, t in 1:n_hours], x[i,j,k,t] <= X[i,j])
    @constraint(model, [i in 1:n_rebal, j in 1:n_rebal, k in 1:K, t in 1:n_hours], v[i,j,k,t] <= X[i,j])
    # Vans must travel sequentially
    @constraint(model, [i in 1:n_rebal, k in 1:K, t in 2:n_hours], sum(x[i,j,k,t] for j in 1:n_rebal) <= sum(v[l,i,k,t-1] for l in 1:n_rebal))
    @constraint(model, [j in 1:n_rebal, k in 1:K, t in 1:n_hours], sum(v[j,l,k,t] for l in 1:n_rebal) <= sum(x[i,j,k,t] for i in 1:n_rebal))
    # Users travel is inferior to the demand and definition of u
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], u[i,j,t] >= d[i,j,t] - w[i,j,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], d[i,j,t] >= w[i,j,t])

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours)+lambda*((sum(D[i,j]*x[i,j,k,t] for i in 1:n_rebal, j in 1:n_rebal, k in 1:K, t in 1:n_hours))+sum(D[j,l]*v[j,l,k,t] for j in 1:n_rebal, l in 1:n_rebal, k in 1:K, t in 1:n_hours)))
    
    # Solve the model
    optimize!(model)
    
    # Print the solution
    println("Objective value: ", objective_value(model))
    return value.(x), value.(v), value.(w), value.(u), value.(y), value.(z), objective_value(model)
end

In [ ]:
# Number of vans
K=3
# Capacity of vans
S=10
# stations for rebalancing 
n_rebal = 50; # Top 50 according to station scores
# Trade-off for multi-objective
lambda=10 # to be tuned according to the objectives scales

In [ ]:
x_1,v_1, w_1,u_1,y_1,z_1,obj_1=solve_sequential_model(K,S,C_new,D_new,X_new,y0_new,d_new,lambda,n_rebal);